In [1]:
import os
import csv

import cv2
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import tensorflow as tf

## Model definition

In [2]:
### Model Parameters
conv_dropout = 0.4
dense_dropout = 0.5
l2_regularization = 0.001

# l1_regularizer = tf.keras.regularizers.l1(0.01)
l2_regularizer = None #tf.keras.regularizers.l2(l2_regularization)

CROP_UP = 50
CROP_DOWN = 30
final_height = 160 - (CROP_UP + CROP_DOWN)

In [3]:
from model import model, deep_model, nvidia_net
mu = 0
sigma = 0.1
init = None #tf.initializers.truncated_normal(mean = mu, stddev = sigma)

params = dict(
    conv_dropout=conv_dropout,
    dense_dropout=dense_dropout,
    kernel_init=init,
    height=final_height
)

def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""

    predictions = nvidia_net(features, mode, params)
    preds = predictions["steering"]


    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    with tf.name_scope('loss'):
            loss = tf.losses.mean_squared_error(
                labels=labels, predictions=tf.squeeze(preds, axis = 1), scope='loss')
            tf.summary.scalar('loss', loss)
    
    # Accuracy    
    with tf.name_scope('mae'):
            mae = tf.metrics.mean_absolute_error(
                labels=labels, predictions=tf.squeeze(preds, axis = 1), name='mae')
            tf.summary.scalar('mae', mae[1])

    # Create a hook to print acc, loss & global step every 100 iter.   
    train_hook_list= []
    train_tensors_log = {'mae': mae[1],
                         'loss': loss}
    train_hook_list.append(tf.train.LoggingTensorHook(
        tensors=train_tensors_log, every_n_iter=100))

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:

        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            optimizer = tf.train.AdamOptimizer(learning_rate = 0.001)
#             optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
#             optimizer = tf.train.AdagradOptimizer(learning_rate=0.001)
            train_op = optimizer.minimize(
                loss=loss,
                global_step=tf.train.get_global_step())
    
        return tf.estimator.EstimatorSpec(
            mode=mode, 
            loss=loss, 
            train_op=train_op, 
            training_hooks=train_hook_list)

    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops={'mae/mae': mae}, evaluation_hooks=None)


In [4]:
# Create the Estimator
classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/tmp/pilotnetVpaper_noaug")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_device_fn': None, '_task_id': 0, '_keep_checkpoint_every_n_hours': 10000, '_global_id_in_cluster': 0, '_is_chief': True, '_tf_random_seed': None, '_keep_checkpoint_max': 5, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_log_step_count_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdb70cb2710>, '_master': '', '_evaluation_master': '', '_train_distribute': None, '_experimental_distribute': None, '_protocol': None, '_save_checkpoints_steps': None, '_model_dir': '/tmp/pilotnetVpaper_noaug', '_service': None, '_task_type': 'worker', '_num_ps_replicas': 0, '_save_checkpoints_secs': 600, '_num_worker_replicas': 1, '_save_summary_steps': 100, '_eval_distribute': None}


## Dataset creation

In [5]:
### folder where the data is
DATASET = "data_mine"

In [6]:
def read_img(name):
    img = name.split('/')[-1]
    return cv2.imread(os.path.join(DATASET,"IMG", img))[...,::-1]

In [7]:
### Load csv as a pandas dataframe
header = ["center", "left", "right", "steering", "throttle", "break", "speed"]
df = pd.read_csv(os.path.join(DATASET, "driving_log.csv"), header=None, names=header)
df.head()

,center,left,right,steering,throttle,break,speed
0,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,0.0,0.0,0,0.000020
1,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,0.0,0.0,0,0.000011
2,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,0.0,0.0,0,0.000011
3,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,0.0,0.0,0,0.000021
4,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,0.0,0.0,0,0.000016


In [8]:
def get_image_path(filename):
    return os.path.join(DATASET,"IMG", filename.split("/")[-1])

In [9]:
### Make paths relative to this jupyter notebook
df["center"] = df.center.apply(get_image_path)
df["left"] = df.left.apply(get_image_path)
df["right"] = df.right.apply(get_image_path)
df["steering"] = df.steering.apply(float)
df.head()

,center,left,right,steering,throttle,break,speed
0,data_mine/IMG/center_2019_02_03_21_24_02_332.jpg,data_mine/IMG/left_2019_02_03_21_24_02_332.jpg,data_mine/IMG/right_2019_02_03_21_24_02_332.jpg,0.0,0.0,0,0.000020
1,data_mine/IMG/center_2019_02_03_21_24_02_398.jpg,data_mine/IMG/left_2019_02_03_21_24_02_398.jpg,data_mine/IMG/right_2019_02_03_21_24_02_398.jpg,0.0,0.0,0,0.000011
2,data_mine/IMG/center_2019_02_03_21_24_02_467.jpg,data_mine/IMG/left_2019_02_03_21_24_02_467.jpg,data_mine/IMG/right_2019_02_03_21_24_02_467.jpg,0.0,0.0,0,0.000011
3,data_mine/IMG/center_2019_02_03_21_24_02_536.jpg,data_mine/IMG/left_2019_02_03_21_24_02_536.jpg,data_mine/IMG/right_2019_02_03_21_24_02_536.jpg,0.0,0.0,0,0.000021
4,data_mine/IMG/center_2019_02_03_21_24_02_605.jpg,data_mine/IMG/left_2019_02_03_21_24_02_605.jpg,data_mine/IMG/right_2019_02_03_21_24_02_605.jpg,0.0,0.0,0,0.000016


In [10]:
### Split train and dev set 
train_samples, validation_samples = train_test_split(df, test_size=0.2)

In [11]:
from data_augmentation import get_seq
NUMBER_AUGMENTATIONS = 3

In [12]:
def single_generator(filenames, labels):
    seq = get_seq(NUMBER_AUGMENTATIONS)
    while 1: # Loop forever so the generator never terminates
        for filename, label in zip(filenames, labels):

            y_train = label
            image = read_img(filename)
            

            # define images for training
            X_train = image[CROP_UP:-CROP_DOWN,:,:].astype(np.float32)
#             X_train = seq.augment_image(X_train).astype(np.float32)
            yield (X_train, y_train)
            
def single_generator_eval(filenames, labels):
#     seq = get_seq(NUMBER_AUGMENTATIONS)
    for filename, label in zip(filenames, labels):

        y_train = label
        image = read_img(filename)


        # define images for training
        X_train = image[CROP_UP:-CROP_DOWN,:,:].astype(np.float32)
#         X_train = seq.augment_image(X_train).astype(np.float32)
        yield (X_train, y_train)

In [13]:
# Get filenames and labels (steerings)
c_filenames = train_samples.center.values
c_labels = train_samples.steering.values

val_filenames = validation_samples.center.values
val_labels = validation_samples.steering.values

In [14]:
train_samples.steering.values

array([ 0.04716982,  0.09433963, -0.03773585, ..., -0.02830189,
       -0.04716982,  0.        ])

In [15]:
# Add side cameras to train set
steer_correction = 0.2

l_filenames = train_samples.left.values
l_labels = train_samples.steering.values + steer_correction

r_filenames = train_samples.right.values
r_labels = train_samples.steering.values - steer_correction

filenames = np.concatenate([c_filenames, l_filenames, r_filenames])
labels = np.concatenate([c_labels, l_labels, r_labels])

In [16]:
train_generator = single_generator(filenames, labels)
eval_generator = single_generator_eval(val_filenames, val_labels)
x,y=next(train_generator)
x.shape, y.shape

((80, 320, 3), ())

In [17]:
# Aux function to crop the images
def get_crop_window(crop_up, crop_down):
    final_height = 160 - (crop_up + crop_down)
    final_width = 320

    return [
        crop_up,
        0,
        final_height,
        final_width,
    ]

In [18]:
# Function that returns the image decoded from jpg and label

def parse_function(image, label):
#     image_string = tf.read_file(filename)

#     # Don't use tf.image.decode_image, or the output shape will be undefined
#     image = tf.image.decode_jpeg(image_string, channels=3)
    
#     image = tf.image.decode_and_crop_jpeg(
#             image_string,
#             crop_window = get_crop_window(CROP_UP, CROP_DOWN),
#             channels = 3
#         )

    # This will convert to float values in [0, 1]
#     image = tf.image.convert_image_dtype(image, tf.float32)

#     image = tf.image.resize_images(image, [160, 320])
    image = tf.cast(image, tf.float32)
    label = tf.cast(label, tf.float32)
    return image, label

### Use of TF DataSet API

In [19]:
# Define parameters
BATCH_SIZE = 32
EPOCHS = 50
steps_per_epoch = len(train_samples)*3//BATCH_SIZE

In [20]:
# Create training dataset
def create_train_dataset():
    dataset = tf.data.Dataset.from_generator(generator=lambda: train_generator,
                                         output_types=(tf.float32, tf.float32))
    dataset = dataset.shuffle(len(train_samples))
#     dataset = dataset.map(parse_function, num_parallel_calls=4)
    dataset = dataset.repeat(EPOCHS)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(1)
    return dataset

In [21]:
# Create dev dataset
def create_eval_dataset():
    val_dataset = tf.data.Dataset.from_generator(generator=lambda: eval_generator,
                                         output_types=(tf.float32, tf.float32))
#     val_dataset = val_dataset.map(parse_function, num_parallel_calls=4)
    val_dataset = val_dataset.repeat(EPOCHS)
    val_dataset = val_dataset.batch(BATCH_SIZE)
    val_dataset = val_dataset.prefetch(1)
    return val_dataset

In [22]:
# # # Get image example for trainig
# sess = tf.Session()
# value = create_train_dataset().make_one_shot_iterator().get_next()
# x,y = sess.run(value)

In [23]:
# print(x.shape, y.shape)
# plt.figure(figsize=(12,80))

# for i, image in enumerate(x):
#     plt.subplot(BATCH_SIZE,1,i+1)
#     plt.imshow(image.astype(np.uint32))

## Training

In [24]:
def serving_input_fn():

    input_image = tf.placeholder(
        dtype=tf.float32,
        shape=[None, None, None, 3],
        name="input_image"
    )

#     images = tf.image.resize_images(input_image, [params.image_height, params.image_width])
    images = input_image
    images = tf.image.crop_to_bounding_box(images, *get_crop_window(CROP_UP, CROP_DOWN))

    images = tf.cast(images, tf.float32)

    return tf.estimator.export.TensorServingInputReceiver(
        features = images,
        receiver_tensors = input_image
    )

In [25]:
exporter = tf.estimator.LatestExporter(
    "test_exporter",
    lambda: serving_input_fn(),
)

In [26]:
# Specs
train_spec = tf.estimator.TrainSpec(
    input_fn=create_train_dataset,
    max_steps=EPOCHS*steps_per_epoch)

eval_spec = tf.estimator.EvalSpec(
    input_fn=create_eval_dataset,
    steps=None,
    exporters=[exporter],
    start_delay_secs=10,  # Start evaluating after 10 sec.
    throttle_secs=30  # Evaluate only every 30 sec
)

In [27]:
tf.estimator.train_and_evaluate(classifier, train_spec, eval_spec)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Calling model_fn.
#######################
## Mask
#######################
Tensor("mask/Relu:0", shape=(?, 3, 33, 64), dtype=float32)
Tensor("mask/Mean:0", shape=(?, 3, 33, 1), dtype=float32)
Tensor("mask/conv2d_transpose:0", shape=(?, 5, 35, 1), dtype=float32)
Tensor("mask/Relu_1:0", shape=(?, 5, 35, 64), dtype=float32)
Tensor("mask/mul:0", shape=(?, 5, 35, 1), dtype=float32)
Tensor("mask/conv2d_transpose_1:0", shape=(?, 7, 37, 1), dtype=float32)
Tensor("mask/Relu_2:0", shape=(?, 7, 37, 48), dtype=float32)
Tensor("mask/mul_1:0", shape=(?, 7, 37, 1), dtype=float32)
Tensor("mask/conv2d_transpose_2:0", shape=(?, 17, 77, 1), dtype=float

INFO:tensorflow:loss = 0.01332936, mae = 0.17946996 (4.595 sec)
INFO:tensorflow:global_step/sec: 21.7971
INFO:tensorflow:loss = 0.02208547, step = 4000 (4.587 sec)
INFO:tensorflow:loss = 0.02208547, mae = 0.17815126 (4.587 sec)
INFO:tensorflow:global_step/sec: 22.1463
INFO:tensorflow:loss = 0.013774188, step = 4100 (4.516 sec)
INFO:tensorflow:loss = 0.013774188, mae = 0.17618588 (4.516 sec)
INFO:tensorflow:global_step/sec: 22.001
INFO:tensorflow:loss = 0.009919314, step = 4200 (4.545 sec)
INFO:tensorflow:loss = 0.009919314, mae = 0.17380594 (4.545 sec)
INFO:tensorflow:global_step/sec: 21.8022
INFO:tensorflow:loss = 0.011454131, step = 4300 (4.586 sec)
INFO:tensorflow:loss = 0.011454131, mae = 0.17183079 (4.586 sec)
INFO:tensorflow:global_step/sec: 21.796
INFO:tensorflow:loss = 0.01128136, step = 4400 (4.588 sec)
INFO:tensorflow:loss = 0.01128136, mae = 0.17006303 (4.589 sec)
INFO:tensorflow:global_step/sec: 21.8376
INFO:tensorflow:loss = 0.01171734, step = 4500 (4.579 sec)
INFO:tensorf

INFO:tensorflow:loss = 0.010671217, step = 8900 (4.583 sec)
INFO:tensorflow:loss = 0.010671217, mae = 0.12998083 (4.584 sec)
INFO:tensorflow:global_step/sec: 21.9495
INFO:tensorflow:loss = 0.010868568, step = 9000 (4.556 sec)
INFO:tensorflow:loss = 0.010868568, mae = 0.12947005 (4.556 sec)
INFO:tensorflow:global_step/sec: 21.9971
INFO:tensorflow:loss = 0.011354065, step = 9100 (4.546 sec)
INFO:tensorflow:loss = 0.011354065, mae = 0.1290142 (4.546 sec)
INFO:tensorflow:global_step/sec: 21.9245
INFO:tensorflow:loss = 0.0076015415, step = 9200 (4.561 sec)
INFO:tensorflow:loss = 0.0076015415, mae = 0.1283456 (4.560 sec)
INFO:tensorflow:global_step/sec: 21.8937
INFO:tensorflow:loss = 0.0073734485, step = 9300 (4.568 sec)
INFO:tensorflow:loss = 0.0073734485, mae = 0.1277423 (4.568 sec)
INFO:tensorflow:global_step/sec: 21.868
INFO:tensorflow:loss = 0.012806535, step = 9400 (4.573 sec)
INFO:tensorflow:loss = 0.012806535, mae = 0.12730247 (4.573 sec)
INFO:tensorflow:global_step/sec: 22.1541
INFO

INFO:tensorflow:Saving dict for global step 12804: global_step = 12804, loss = 0.008558633, mae/mae = 0.07748373
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 12804: /tmp/pilotnetVpaper_noaug/model.ckpt-12804
INFO:tensorflow:Calling model_fn.
#######################
## Mask
#######################
Tensor("mask/Relu:0", shape=(?, 3, 33, 64), dtype=float32)
Tensor("mask/Mean:0", shape=(?, 3, 33, 1), dtype=float32)
Tensor("mask/conv2d_transpose:0", shape=(?, 5, 35, 1), dtype=float32)
Tensor("mask/Relu_1:0", shape=(?, 5, 35, 64), dtype=float32)
Tensor("mask/mul:0", shape=(?, 5, 35, 1), dtype=float32)
Tensor("mask/conv2d_transpose_1:0", shape=(?, 7, 37, 1), dtype=float32)
Tensor("mask/Relu_2:0", shape=(?, 7, 37, 48), dtype=float32)
Tensor("mask/mul_1:0", shape=(?, 7, 37, 1), dtype=float32)
Tensor("mask/conv2d_transpose_2:0", shape=(?, 17, 77, 1), dtype=float32)
Tensor("mask/Relu_3:0", shape=(?, 17, 77, 36), dtype=float32)
Tensor("mask/mul_2:0", shape=(?, 17, 77, 1), dtype

INFO:tensorflow:global_step/sec: 21.8556
INFO:tensorflow:loss = 0.010169713, step = 16400 (4.576 sec)
INFO:tensorflow:loss = 0.010169713, mae = 0.10505295 (4.576 sec)
INFO:tensorflow:global_step/sec: 21.9344
INFO:tensorflow:loss = 0.00580802, step = 16500 (4.559 sec)
INFO:tensorflow:loss = 0.00580802, mae = 0.104781955 (4.560 sec)
INFO:tensorflow:global_step/sec: 21.7945
INFO:tensorflow:loss = 0.006579245, step = 16600 (4.588 sec)
INFO:tensorflow:loss = 0.006579245, mae = 0.10455832 (4.588 sec)
INFO:tensorflow:global_step/sec: 21.8498
INFO:tensorflow:loss = 0.007202836, step = 16700 (4.576 sec)
INFO:tensorflow:loss = 0.007202836, mae = 0.1043634 (4.576 sec)
INFO:tensorflow:global_step/sec: 21.8029
INFO:tensorflow:loss = 0.005812566, step = 16800 (4.587 sec)
INFO:tensorflow:loss = 0.005812566, mae = 0.104112096 (4.587 sec)
INFO:tensorflow:global_step/sec: 21.9076
INFO:tensorflow:loss = 0.010117166, step = 16900 (4.565 sec)
INFO:tensorflow:loss = 0.010117166, mae = 0.10396286 (4.565 sec)

INFO:tensorflow:global_step/sec: 21.8931
INFO:tensorflow:loss = 0.005446847, step = 21300 (4.568 sec)
INFO:tensorflow:loss = 0.005446847, mae = 0.09709007 (4.568 sec)
INFO:tensorflow:global_step/sec: 22.2334
INFO:tensorflow:loss = 0.0044723675, step = 21400 (4.498 sec)
INFO:tensorflow:loss = 0.0044723675, mae = 0.096897066 (4.497 sec)
INFO:tensorflow:global_step/sec: 21.8208
INFO:tensorflow:loss = 0.0036470583, step = 21500 (4.583 sec)
INFO:tensorflow:loss = 0.0036470583, mae = 0.09665819 (4.583 sec)
INFO:tensorflow:global_step/sec: 21.909
INFO:tensorflow:loss = 0.00657095, step = 21600 (4.564 sec)
INFO:tensorflow:loss = 0.00657095, mae = 0.09650881 (4.565 sec)
INFO:tensorflow:global_step/sec: 21.846
INFO:tensorflow:loss = 0.008719059, step = 21700 (4.578 sec)
INFO:tensorflow:loss = 0.008719059, mae = 0.09637527 (4.578 sec)
INFO:tensorflow:global_step/sec: 21.9577
INFO:tensorflow:loss = 0.0076744654, step = 21800 (4.553 sec)
INFO:tensorflow:loss = 0.0076744654, mae = 0.096238 (4.553 se

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-13-14:51:24
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/pilotnetVpaper_noaug/model.ckpt-25803
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-13-14:51:25
INFO:tensorflow:Saving dict for global step 25803: global_step = 25803, loss = 0.0, mae/mae = 0.0
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 25803: /tmp/pilotnetVpaper_noaug/model.ckpt-25803
INFO:tensorflow:Calling model_fn.
#######################
## Mask
#######################
Tensor("mask/Relu:0", shape=(?, 3, 33, 64), dtype=float32)
Tensor("mask/Mean:0", shape=(?, 3, 33, 1), dtype=float32)
Tensor("mask/conv2d_transpose:0", shape=(?, 5, 35, 1), dtype=float32)
Tensor("mask/Relu_1:0", shape=(?, 5, 35, 64), dtype=float32)
Tensor("mask/mul:0", shape=(?, 5, 35, 1), dtype=float32)
Tensor("mask/conv2d_transpose_

INFO:tensorflow:global_step/sec: 21.9283
INFO:tensorflow:loss = 0.0077649653, step = 29400 (4.561 sec)
INFO:tensorflow:loss = 0.0077649653, mae = 0.08741332 (4.560 sec)
INFO:tensorflow:global_step/sec: 21.8612
INFO:tensorflow:loss = 0.0044892365, step = 29500 (4.574 sec)
INFO:tensorflow:loss = 0.0044892365, mae = 0.08729319 (4.575 sec)
INFO:tensorflow:global_step/sec: 21.8522
INFO:tensorflow:loss = 0.004961323, step = 29600 (4.576 sec)
INFO:tensorflow:loss = 0.004961323, mae = 0.08719544 (4.576 sec)
INFO:tensorflow:global_step/sec: 22.0297
INFO:tensorflow:loss = 0.006470417, step = 29700 (4.540 sec)
INFO:tensorflow:loss = 0.006470417, mae = 0.08711146 (4.540 sec)
INFO:tensorflow:global_step/sec: 22.0363
INFO:tensorflow:loss = 0.0036615734, step = 29800 (4.537 sec)
INFO:tensorflow:loss = 0.0036615734, mae = 0.086988 (4.538 sec)
INFO:tensorflow:global_step/sec: 21.9001
INFO:tensorflow:loss = 0.0061811814, step = 29900 (4.567 sec)
INFO:tensorflow:loss = 0.0061811814, mae = 0.08689794 (4.5

({'global_step': 30750, 'loss': 0.0, 'mae/mae': 0.0},
 [b'/tmp/pilotnetVpaper_noaug/export/test_exporter/1552488914'])

In [28]:
classifier.export_savedmodel(
    "export/nvidia",
    lambda: serving_input_fn()
)

INFO:tensorflow:Calling model_fn.
#######################
## Mask
#######################
Tensor("mask/Relu:0", shape=(?, 3, 33, 64), dtype=float32)
Tensor("mask/Mean:0", shape=(?, 3, 33, 1), dtype=float32)
Tensor("mask/conv2d_transpose:0", shape=(?, 5, 35, 1), dtype=float32)
Tensor("mask/Relu_1:0", shape=(?, 5, 35, 64), dtype=float32)
Tensor("mask/mul:0", shape=(?, 5, 35, 1), dtype=float32)
Tensor("mask/conv2d_transpose_1:0", shape=(?, 7, 37, 1), dtype=float32)
Tensor("mask/Relu_2:0", shape=(?, 7, 37, 48), dtype=float32)
Tensor("mask/mul_1:0", shape=(?, 7, 37, 1), dtype=float32)
Tensor("mask/conv2d_transpose_2:0", shape=(?, 17, 77, 1), dtype=float32)
Tensor("mask/Relu_3:0", shape=(?, 17, 77, 36), dtype=float32)
Tensor("mask/mul_2:0", shape=(?, 17, 77, 1), dtype=float32)
Tensor("mask/conv2d_transpose_3:0", shape=(?, 38, 158, 1), dtype=float32)
Tensor("mask/Relu_4:0", shape=(?, 38, 158, 36), dtype=float32)
Tensor("mask/mul_3:0", shape=(?, 38, 158, 1), dtype=float32)
Tensor("mask/conv2d_

b'export/nvidia/1552488915'